## Import libraries

In [11]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
import pandas as pd
import ast

## Setup Elastic search configuration

In [8]:
es = Elasticsearch(
    "http://localhost:9200"
)

In [9]:
es.ping()

True

## Prepare the data

In [12]:
def merge_details(dataframe):
    dataframe['merge_details'] = dataframe['title'] + " " + dataframe['updated_genres'] + " " + dataframe['overview'] + " " +dataframe['tagline'] + " " +dataframe['updated_spoken_languages']
    return dataframe

def retireve_genres(items):
    return [item['name'] for item in items] 

dataframe1 = pd.read_csv('docs/movies_metadata.csv',sep = ',')
dataframe2 = pd.read_csv('docs/ratings_small.csv',sep=',')
dataframe1['bool_id'] = dataframe1['id'].str.isdigit()
dataframe1 = dataframe1.drop(dataframe1[dataframe1['bool_id'] == False].index,axis=0)
dataframe1 = dataframe1.reset_index(drop=True)
dataframe1['id'] = dataframe1['id'].apply(lambda x: int(x))
dataframe2.rename(columns={'movieId':'id'}, inplace=True)
merge_dataframe = pd.merge(
    dataframe1, 
    dataframe2, 
    on='id'
)[['id','title','rating','genres','overview','tagline','spoken_languages']].dropna().reset_index(drop=True)
merge_dataframe['genres'] = merge_dataframe['genres'].apply(lambda x: ast.literal_eval(x))
merge_dataframe['spoken_languages'] = merge_dataframe['spoken_languages'].apply(lambda x: ast.literal_eval(x))
merge_dataframe['genres'] = merge_dataframe['genres'].apply(retireve_genres)
merge_dataframe['spoken_languages'] = merge_dataframe['spoken_languages'].apply(retireve_genres)
merge_dataframe['updated_genres'] =  merge_dataframe['genres'].apply(lambda x : ' '.join(x))
merge_dataframe['updated_spoken_languages'] = merge_dataframe['spoken_languages'].apply(lambda x : ' '.join(x))
final_data = merge_details(merge_dataframe)

C:\Users\User\AppData\Local\Temp\ipykernel_9292\3817152956.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe1 = pd.read_csv('docs/movies_metadata.csv',sep = ',')


In [47]:
df = final_data.loc[final_data['merge_details'].drop_duplicates(keep='first').index].reset_index(drop=True)
df.head()

,id,title,rating,genres,overview,tagline,spoken_languages,updated_genres,updated_spoken_languages,merge_details
0,949,Heat,3.5,"[Action, Crime, Drama, Thriller]","Obsessive master thief, Neil McCauley leads a ...",A Los Angeles Crime Saga,"[English, Español]",Action Crime Drama Thriller,English Español,Heat Action Crime Drama Thriller Obsessive mas...
1,710,GoldenEye,1.0,"[Adventure, Action, Thriller]",James Bond must unmask the mysterious head of ...,No limits. No fears. No substitutes.,"[English, Pусский, Español]",Adventure Action Thriller,English Pусский Español,GoldenEye Adventure Action Thriller James Bond...
2,1408,Cutthroat Island,1.0,"[Action, Adventure]","Morgan Adams and her slave, William Shaw, are ...",The Course Has Been Set. There Is No Turning B...,"[English, Latin]",Action Adventure,English Latin,Cutthroat Island Action Adventure Morgan Adams...
3,524,Casino,2.0,"[Drama, Crime]",The life of the gambling paradise – Las Vegas ...,No one stays at the top forever.,[English],Drama Crime,English,Casino Drama Crime The life of the gambling pa...
4,4584,Sense and Sensibility,5.0,"[Drama, Romance]","Rich Mr. Dashwood dies, leaving his second wif...",Lose your heart and come to your senses.,[English],Drama Romance,English,Sense and Sensibility Drama Romance Rich Mr. D...


## Convert the embeddings of the fields

In [15]:
model = SentenceTransformer("all-mpnet-base-v2")

.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 588kB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 190kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 6.75MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 116kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 188kB/s]
pytorch_model.bin: 100%|██████████| 438M/438M [00:24<00:00, 17.5MB/s] 
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 52.2kB/s]
special_tokens_map.json: 100%|██████████| 239/239 [00:00<00:00, 239kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 655kB/s]
tokenizer_config.json: 100%|██████████| 363/363 [00:00<00:00, 181kB/s]
train_script.py: 100%|██████████| 13.1k/13.1k [00:00<00:00, 13.1MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 560kB/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 175kB/s]


In [54]:
df['embeddings'] = df['merge_details'].apply(lambda x: model.encode(x))

In [62]:
df.head()

,id,title,rating,genres,overview,tagline,spoken_languages,updated_genres,updated_spoken_languages,merge_details,embeddings
0,949,Heat,3.5,"[Action, Crime, Drama, Thriller]","Obsessive master thief, Neil McCauley leads a ...",A Los Angeles Crime Saga,"[English, Español]",Action Crime Drama Thriller,English Español,Heat Action Crime Drama Thriller Obsessive mas...,"[-0.042521507, 0.020714978, 0.005733385, 0.023..."
1,710,GoldenEye,1.0,"[Adventure, Action, Thriller]",James Bond must unmask the mysterious head of ...,No limits. No fears. No substitutes.,"[English, Pусский, Español]",Adventure Action Thriller,English Pусский Español,GoldenEye Adventure Action Thriller James Bond...,"[0.0058759227, -0.025645023, 0.04358933, -0.00..."
2,1408,Cutthroat Island,1.0,"[Action, Adventure]","Morgan Adams and her slave, William Shaw, are ...",The Course Has Been Set. There Is No Turning B...,"[English, Latin]",Action Adventure,English Latin,Cutthroat Island Action Adventure Morgan Adams...,"[-0.04299491, 0.03970336, 0.012585637, -0.0166..."
3,524,Casino,2.0,"[Drama, Crime]",The life of the gambling paradise – Las Vegas ...,No one stays at the top forever.,[English],Drama Crime,English,Casino Drama Crime The life of the gambling pa...,"[-0.02989307, 0.06692077, 0.009279962, 0.02446..."
4,4584,Sense and Sensibility,5.0,"[Drama, Romance]","Rich Mr. Dashwood dies, leaving his second wif...",Lose your heart and come to your senses.,[English],Drama Romance,English,Sense and Sensibility Drama Romance Rich Mr. D...,"[-0.023869524, -0.047999404, -0.009230304, 0.0..."


In [ ]:
from indexMapping import indexMapping

es.indices.create(index="elastic-demo", body=indexMapping)